# Importing Libraries

In [1]:
import spacy
from spacy.matcher import Matcher
import re
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import spacy
from pyresparser import ResumeParser
import numpy as np
import re
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Harshpreet Singh\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


True

In [2]:
import pandas as pd
df = pd.read_csv('processed_resume_data.csv')
df = df.iloc[:,1:]
df.head()

,Resume Text,Clean Text
0,ANIL KUMAR MADDUKURI SQL & MSBI Developer ...,anil kumar maddukuri sql msbi developer summar...
1,Aradhana Tripathi Current Location: Gachibowli...,aradhana tripathi current location gachibowli ...
2,BUDDHA VAMSI ...,buddha vamsi career objective have 2.11 year i...
3,Chinna Subbarayudu M DOB: 06th March 1994 Nati...,chinna subbarayudu m dob 06th march 1994 natio...
4,Name : Gopi Krishna Reddy ...,name gopi krishna reddy professional summery w...


# Extracting Names

In [4]:
# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)
name = []
def extract_name(resume_text, isCapital = False):
    if isCapital:
        return re.findall(r"[A-Z][A-Z]+\s+[A-Z][A-Z]+", resume_text)[0]
    else:
        nlp_text = nlp(resume_text)
        
        # First name and Last name are always Proper Nouns
        pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
        
        matcher.add('NAME', [pattern], on_match=None)
        
        matches = matcher(nlp_text)
        
        for match_id, start, end in matches:
            span = nlp_text[start:end]
            return span.text

# Calling function
for text in df['Resume Text']:
    name.append(extract_name(text, isCapital=False))

C:\Program Files\Python39\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [5]:
details_df = pd.DataFrame(name, columns=['Name'])

In [78]:
details_df.head()

,Name
0,ANIL KUMAR
1,Aradhana Tripathi
2,Database Engineer
3,Chinna Subbarayudu
4,Gopi Krishna


# Extracting Number

In [6]:
# +911234567890 or 1234567890 or +91-1234567890 or +91 1234567890
number = []
def extract_mobile_number(text):
    y = re.findall(r'[+]91[0-9]+|[0-9]{10}|[+?][0-9]+-[0-9]+| [+?][0-9]+\s[0-9]+',text)
    return y

# Calling Function
for text in df['Resume Text']:
    number.append(extract_mobile_number(text))

In [7]:
details_df['Number'] = number
details_df.head()

# Extracting Email

In [8]:
email = []
def extract_email(text):
    email = re.findall("[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", text)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None

# Calling function
for text in df['Resume Text']:
    email.append(extract_email(text))

In [82]:
details_df['Email ID'] = email
details_df.head()

,Name,Number,Email ID
0,ANIL KUMAR,[+911234567890],ijayawadaabc@xyz.com
1,Aradhana Tripathi,[+911234567890],niversity.abc@xyz.com
2,Database Engineer,[+911234567890],knowledge.abc@xyz.com
3,Chinna Subbarayudu,[+911234567890],2abc@xyz.com
4,Gopi Krishna,[+911234567890],annada.abc@xyz.com


# Extracting links

In [10]:
links = []
def extract_links(text):
    y = re.findall(r'(https://(www.)?[a-z]+.com(/in)?/[a-z0-9]+)',text)    # Pattern to extract links
    links = []
    for strings in y:
        for link in strings:
            if len(link) > 4:
                links.append(link)
    return links

# Calling function
for text in df['Resume Text']:
    links.append(extract_links(text))

In [84]:
details_df['Links'] = links
details_df.head()

,Name,Number,Email ID,Links
0,ANIL KUMAR,[+911234567890],ijayawadaabc@xyz.com,"[https://www.linkedin.com/fake, https://www.gi..."
1,Aradhana Tripathi,[+911234567890],niversity.abc@xyz.com,"[https://www.linkedin.com/fake, https://www.gi..."
2,Database Engineer,[+911234567890],knowledge.abc@xyz.com,"[https://www.linkedin.com/fake, https://www.gi..."
3,Chinna Subbarayudu,[+911234567890],2abc@xyz.com,"[https://www.linkedin.com/fake, https://www.gi..."
4,Gopi Krishna,[+911234567890],annada.abc@xyz.com,"[https://www.linkedin.com/fake, https://www.gi..."


# Extracting Education

In [7]:
# details_df = pd.read_csv('FinalDF.csv')
# details_df = df.iloc[:,1:]

In [12]:
import spacy
from nltk.corpus import stopwords

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))

# Education Degrees
EDUCATION = [
            'BE','B.E.', 'B.E', 'BS', 'B.S', 'B.SC', 'B E', 'B. E.','B. E','B S','B. S','B. SC'
            'ME', 'M.E', 'M.E.', 'MS', 'M.S', 'B-TECH','M-TECH','M E', 'M. E', 'M. E.', 'M S', 'M. S',
            'BTECH', 'B.TECH', 'M.TECH', 'MTECH','B TECH', 'B. TECH', 'M. TECH', 'M TECH',
            'B. TECH','M. TECH','B TECH','M TECH',
            'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII',
            'Bachelor of Technology','Senior Secondary'
            'BBA','B.B.A.','BCA','B.C.A.','BA','B.A.',
            'B.COM','B.ED','L.L.B.','LLB','LLM','L.L.M.',
            'MBA','M.B.A.','MCA','M.C.A.','MS','M.S.','MD',
            'M.D.','NDA','N.D.A.','PHD','PGDM','P.G.D.M.'
        ]


education = []
def extract_education(resume_text):
    nlp_text = nlp(resume_text)

    # Sentence Tokenizer
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]

    edu = {}
    # Extract education degree
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                edu[tex] = text + nlp_text[index]   # Storing degree in a dictionary

    # Extract year
    education = []
    if edu:
        for key in edu.keys():
            year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])   # Search for Year of Passing
            if year:
                education.append((key, ''.join(year[0])))
            else:
                education.append(key)
        return education
    else:
        EDU_PATTERN = re.findall(r'Bachelors? \D+|Masters? \D+',resume_text)    # If no degree is found, try searching for Bachelors or Masters in xyz
        return EDU_PATTERN
        

# Calling function
for text in df['Resume Text']:
    education.append(extract_education(text))

C:\Program Files\Python39\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [86]:
details_df['Education'] = education
details_df.head()

,Name,Number,Email ID,Links,Education
0,ANIL KUMAR,[+911234567890],ijayawadaabc@xyz.com,"[https://www.linkedin.com/fake, https://www.gi...","[(MS, 2016), BTech]"
1,Aradhana Tripathi,[+911234567890],niversity.abc@xyz.com,"[https://www.linkedin.com/fake, https://www.gi...","[MS, MCA]"
2,Database Engineer,[+911234567890],knowledge.abc@xyz.com,"[https://www.linkedin.com/fake, https://www.gi...","[MS, (SSC, 2011)]"
3,Chinna Subbarayudu,[+911234567890],2abc@xyz.com,"[https://www.linkedin.com/fake, https://www.gi...",[MS]
4,Gopi Krishna,[+911234567890],annada.abc@xyz.com,"[https://www.linkedin.com/fake, https://www.gi...",[Bachelor of Degree from JNTU - K University in ]


In [87]:
details_df['Education'].isnull().sum()

0

In [16]:
# details_df.to_csv('FinalDF.csv')

# Skills

In [44]:
from pyresparser import ResumeParser
mypath = 'Resumes2'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
skills = []

# Extracting skills
for file in path:
    skills.append(ResumeParser(f'/content/drive/My Drive/Resumes2/{file}').get_extracted_data()['skills'])  
skills = np.array(skills).reshape(-1,1)
df = pd.DataFrame(skills, columns=['Skills'])
df.to_csv('SkillsDF.csv')

C:\Program Files\Python39\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
C:\Program Files\Python39\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [89]:
skills_df = pd.read_csv('SkillsDF.csv')
skills_df = skills_df.iloc[:,1:]
details_df['Skills'] = skills_df['Skills']
details_df.head()

,Name,Number,Email ID,Links,Education,Skills
0,ANIL KUMAR,[+911234567890],ijayawadaabc@xyz.com,"[https://www.linkedin.com/fake, https://www.gi...","[(MS, 2016), BTech]","['Sql', 'Debugging', 'Technical', 'Windows', '..."
1,Aradhana Tripathi,[+911234567890],niversity.abc@xyz.com,"[https://www.linkedin.com/fake, https://www.gi...","[MS, MCA]","['Sql', 'Acquisition', 'Stakeholder management..."
2,Database Engineer,[+911234567890],knowledge.abc@xyz.com,"[https://www.linkedin.com/fake, https://www.gi...","[MS, (SSC, 2011)]","['Sql', 'Servers', 'Technical', 'Programming',..."
3,Chinna Subbarayudu,[+911234567890],2abc@xyz.com,"[https://www.linkedin.com/fake, https://www.gi...",[MS],"['Policies', 'Sql', 'Web services', 'Prototypi..."
4,Gopi Krishna,[+911234567890],annada.abc@xyz.com,"[https://www.linkedin.com/fake, https://www.gi...",[Bachelor of Degree from JNTU - K University in ],"['Specifications', 'Technical', 'Communication..."


In [90]:
details_df.isnull().sum()

Name         0
Number       0
Email ID     0
Links        0
Education    0
Skills       0
dtype: int64

In [91]:
# details_df.to_csv('FinalDF.csv')

In [5]:
import pandas as pd
details_df = pd.read_csv('FinalDF.csv')
df = pd.read_csv('processed_resume_data.csv')
details_df = details_df.iloc[:,1:]
df = df.iloc[:,1:]

In [48]:
import re
import nltk
from nltk.tokenize import sent_tokenize

In [49]:
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Harshpreet Singh\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


True

# Extracting University Name

In [50]:
uni_df = pd.read_csv('List of Universities.csv')
uni_df = uni_df.iloc[:,1:]

In [52]:
f_uni = []

for k in range(len(df)):    # Loop to input Resumes one at a time
    university_name = []
    lines = sent_tokenize(df['Resume Text'][k])          # Tokenizing text into sentences
    for sentence in lines:                               # Loop to run through each sentence
            if re.search('university',sentence.lower()) or re.search('education',sentence.lower()) or re.search('qualifications',sentence.lower()): # Search for words like education in sentence
            # print(sentence)
                if re.search(r'\s{3,5}',sentence):  # If sentence has more than 3 spaces between words then split them
                    sens = re.split(r'\s{3,5}', sentence)
                    for i in sens:
                        if re.search('university',i.lower()) or re.search('education',i.lower()) or re.search('qualifications',i.lower()):  # Search for keywords like education in splitted sentences
                            edu_sen = i
                            # print(i[:100])
                            for j in uni_df['Name of University']:  # Loop to run through each university name
                                if re.search(j.lower(),edu_sen.lower()):
                                    university_name.append(j)   # If university is found, append it.
                                    # print(j)

                else:                               # If there are no 3-5 spaces found, then run following
                    edu_sen = sentence
                    for j in uni_df['Name of University']:      # Loop to run through each university name
                        if re.search(j.lower(),edu_sen.lower()):
                            university_name.append(j)           # If university is found, append it.
            

                if university_name:
                    continue
                else:
                    # print(sentence)
                    university_name.append(sentence)    # If particular university name not found, append whole sentence with keywords like education.

    # Remove duplicates    
    for i in range(len(university_name)-1):
        if university_name[i] == university_name[i+1]:
            del university_name[i]
    
    # Finally, appending all found universities and moving onto next resume
    f_uni.append(university_name)

In [223]:
len(f_uni)

78

In [229]:
def clean_data(text):
    text = re.sub('\s+',' ',str(text)) # Remove Extra Spaces
    text = re.sub('abc@xyz.com +911234567890 https://www.linkedin.com/fake https://www.github.com/fake','',str(text)) # Remove gibberish 
    return text

# Calling function
uni_list = []
for ele in f_uni:
    uni_list.append(clean_data(ele))

In [231]:
len(uni_list)

78

In [232]:
details_df['University Name'] = uni_list
details_df.head(10)

,Name,Number,Email ID,Links,Education,Skills,University Name,Years of Experience
0,ANIL KUMAR,['+911234567890'],ijayawadaabc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...","[('MS', '2016'), 'BTech']","['Sql', 'Debugging', 'Technical', 'Windows', '...",['Velagapudi siddhartha engineering college'],2.4
1,Aradhana Tripathi,['+911234567890'],niversity.abc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...","['MS', 'MCA']","['Sql', 'Acquisition', 'Stakeholder management...","['AKS University', ' IIIT ']",4
2,Database Engineer,['+911234567890'],knowledge.abc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...","['MS', ('SSC', '2011')]","['Sql', 'Servers', 'Technical', 'Programming',...",['Create ETL Jobs And Monitoring The Jobs ACAD...,NaN
3,Chinna Subbarayudu,['+911234567890'],2abc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...",['MS'],"['Policies', 'Sql', 'Web services', 'Prototypi...",['Yogi Vemana University'],5.1
4,Gopi Krishna,['+911234567890'],annada.abc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...",['Bachelor of Degree from JNTU - K University ...,"['Specifications', 'Technical', 'Communication...",['JNTU'],3+
5,Hari Krishna,['+911234567890'],11iabc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...","[('MCA', '2013')]","['Hardware', 'Web services', 'Prototyping', 'S...","['JNTU University', 'JNTU']",3.10
6,Harikrishna Akula,['+911234567890'],andyala.abc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...","['Ms', 'MS']","['Sql', 'Web services', 'Workflow', 'Specifica...",['EDUCATION: Bachelore in Civil Engineering Fr...,5.2
7,Career Objective,['+911234567890'],basis.abc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...",['Be'],"['Policies', 'System', 'Cloud', 'Engagement', ...",[],NaN
8,G Himaja,['+911234567890'],imajaabc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...",NaN,"['End user', 'Technical', 'Schedule', 'Windows...","['Education: Completed Degree (BSc.', 'S V Uni...",3
9,Ravali P,['+911234567890'],angaloreabc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...",['BE'],"['Sql', 'Sdlc', 'Technical', 'Test cases', 'En...",['E D U C A T ION E D U C A T ION Degree/Cours...,NaN


In [148]:
# details_df.to_csv('FinalDF.csv')

In [149]:
details_df = pd.read_csv('FinalDF.csv')
details_df = details_df.iloc[:,1:]

# Extracting Years of Experience

In [1]:
import numpy as np

In [56]:
def extract_years_of_experience(text):
    lines = sent_tokenize(text)             # Sentence Tokenization
    experience = []
    for sentence in lines:
        if re.search('experience',sentence.lower()):        # Search for 'experience' keyword in sentence
            sen_tokenized = nltk.word_tokenize(sentence)
            tagged = nltk.pos_tag(sen_tokenized)            # POS Tagging
            entities = nltk.chunk.ne_chunk(tagged)          # Structuring data into tree
            for subtree in entities.subtrees():
                for leaf in subtree.leaves():
                    # print(leaf)
                    if leaf[1] == 'CD':                     # Search for numerical values in the sentence
                        experience.append(leaf[0])
                        
    exp = []
    for ele in experience:
        if len(ele) <= 3 or (len(ele) <= 4 and ele[-1] == '0' 
                                and ele not in ('2020','2010','2000')):       # Finding relevant numerical value for experience
            exp.append(ele)
    if exp:
        return exp[0]
    else:
        return np.nan


# Calling function
years_of_experience = []
for i in range(len(df)):
    years_of_experience.append(extract_years_of_experience(df['Resume Text'][i]))

In [7]:
details_df['Years of Experience'] = years_of_experience
details_df.head(10)

,Name,Number,Email ID,Links,Education,Skills,University Name,Years of Experience
0,ANIL KUMAR,['+911234567890'],ijayawadaabc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...","[('MS', '2016'), 'BTech']","['Sql', 'Debugging', 'Technical', 'Windows', '...",['Velagapudi siddhartha engineering college'],2.4
1,Aradhana Tripathi,['+911234567890'],niversity.abc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...","['MS', 'MCA']","['Sql', 'Acquisition', 'Stakeholder management...","['AKS University', ' IIIT ']",4
2,Database Engineer,['+911234567890'],knowledge.abc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...","['MS', ('SSC', '2011')]","['Sql', 'Servers', 'Technical', 'Programming',...",['Create ETL Jobs And Monitoring The Jobs ACAD...,NaN
3,Chinna Subbarayudu,['+911234567890'],2abc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...",['MS'],"['Policies', 'Sql', 'Web services', 'Prototypi...",['Yogi Vemana University'],5.1
4,Gopi Krishna,['+911234567890'],annada.abc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...",['Bachelor of Degree from JNTU - K University ...,"['Specifications', 'Technical', 'Communication...",['JNTU'],3+
5,Hari Krishna,['+911234567890'],11iabc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...","[('MCA', '2013')]","['Hardware', 'Web services', 'Prototyping', 'S...","['JNTU University', 'JNTU']",3.10
6,Harikrishna Akula,['+911234567890'],andyala.abc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...","['Ms', 'MS']","['Sql', 'Web services', 'Workflow', 'Specifica...",['EDUCATION: Bachelore in Civil Engineering Fr...,5.2
7,Career Objective,['+911234567890'],basis.abc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...",['Be'],"['Policies', 'System', 'Cloud', 'Engagement', ...",NaN,NaN
8,G Himaja,['+911234567890'],imajaabc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...",NaN,"['End user', 'Technical', 'Schedule', 'Windows...","['Education: Completed Degree (BSc.', 'S V Uni...",3
9,Ravali P,['+911234567890'],angaloreabc@xyz.com,"['https://www.linkedin.com/fake', 'https://www...",['BE'],"['Sql', 'Sdlc', 'Technical', 'Test cases', 'En...",['E D U C A T ION E D U C A T ION Degree/Cours...,NaN


In [233]:
# Replacing empty lists with NaN
details_df.replace('[]', np.nan, inplace=True)

In [8]:
details_df.isnull().sum()

Name                    0
Number                  0
Email ID                0
Links                   0
Education              11
Skills                  0
University Name         9
Years of Experience     9
dtype: int64

In [59]:
# details_df.to_csv('FinalDF.csv')